# Intake
Author: [Jason Boutte](https://github.com/jasonb5)

Date: 08/17/23

Last Updated: 08/17/23

Sources:
- https://github.com/intake/intake
- https://github.com/esgf-nimbus/nimbus/tree/main/dockerfiles/minimal-notebook/intake_es

Documentation:
- https://intake.readthedocs.io/en/latest/

## Changelog

## Overview
In this notebook we'll use the `intake_es` package to search and access local data on the Nimbus cluster.

The `intake_es` package provides a new Catalog that can query an ElasticSearch (ES) instance that we've populated with the metadata of locally available data. The package also includes everything required to access these ES indexes.

The following projects have been indexed:
- CMIP6
- CMIP5

**Warning:** The `intake_es` package is *beta* and can change.

In [1]:
import intake
import json

import warnings

warnings.simplefilter('ignore')

## 1. Look at the available indexes
The local catalogs are available under `intake.cat.local`.

In [2]:
list(intake.cat.local)

['cmip6', 'cmip5']

## 2. Using an catalog
Here we'll use the CMIP6 catalog and start looking at the index.

In [3]:
cmip6 = intake.cat.local.cmip6
cmip6

## 3. Display the total number of datasets indexed

In [4]:
cmip6.count()

2023-08-24 01:33:15,397 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_count [status:200 duration:0.009s]


6020752

## 4. List the index fields
This will list the searchable fields of the index.

In [5]:
cmip6.fields

2023-08-24 01:33:15,404 [INFO]: _transport.py(perform_request:336) >> GET https://jupyterhub-es-http:9200/cmip6/_mapping [status:200 duration:0.002s]


['activity_drs',
 'experiment_id',
 'grid_label',
 'institution_id',
 'member_id',
 'mip_era',
 'path',
 'source_id',
 'table_id',
 'variable_id',
 'version']

## 5. List the top values of a field
This will list the top (10) values of a field along with the number of matches.

In [6]:
cmip6.top('activity_drs')

2023-08-24 01:33:15,418 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.003s]


,activity_drs,count
0,ta,85019
1,ua,84888
2,psl,81208
3,tas,78834
4,pr,76706
5,va,74407
6,zg,67312
7,rsds,66617
8,hus,63457
9,sfcWind,62597


## 6. Search for some data
Calling `search` will apply the query and retrun a new Catalog containing the results. With the new catalog you can explore the results or continue to refine your search.

**WARNING:** By default a search will return all results, this can take a long time if there are a large number of matches.

#### Notes:
- It's recommended to use `top` methods to find some starting values to search for.
- Passing a list to a field will return results matching either value, e.g. `variable_id=['tas', 'pr']` will return matches with either `tas` or `pr`.
- You can negate a value by prefixing it with `!`, e.g. `variable_id='!tas'` will return all variable_id's except `tas`.

In [7]:
result = cmip6.search(
    activity_drs='CMIP',
    institution_id=['MIROC', 'NCAR'],
    source_id=['MIROC6', 'CESM2'],
    experiment_id='historical',
    variable_id='tas'
)
result

2023-08-24 01:33:15,438 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.006s]
2023-08-24 01:33:15,442 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.003s]


,mip_era,activity_drs,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,path
0,CMIP6,CMIP,MIROC,MIROC6,historical,r2i1p1f1,3hr,tas,gn,v20190912,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
1,CMIP6,CMIP,MIROC,MIROC6,historical,r2i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
2,CMIP6,CMIP,MIROC,MIROC6,historical,r2i1p1f1,Amon,tas,gn,v20181212,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
3,CMIP6,CMIP,MIROC,MIROC6,historical,r35i1p1f1,day,tas,gn,v20200519,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
4,CMIP6,CMIP,MIROC,MIROC6,historical,r35i1p1f1,Amon,tas,gn,v20200519,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
...,...,...,...,...,...,...,...,...,...,...,...
127,CMIP6,CMIP,NCAR,CESM2,historical,r4i1p1f1,day,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
128,CMIP6,CMIP,NCAR,CESM2,historical,r4i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
129,CMIP6,CMIP,NCAR,CESM2,historical,r3i1p1f1,day,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
130,CMIP6,CMIP,NCAR,CESM2,historical,r3i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...


## 7. Further refine the results
Since the above operation returns a catalog, we can call `search` again to further refine the results.

In [8]:
result.search(member_id='r1i1p1f1')

,mip_era,activity_drs,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,path
0,CMIP6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,3hr,tas,gn,v20190912,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
1,CMIP6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
2,CMIP6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,Amon,tas,gn,v20181212,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
3,CMIP6,CMIP,NCAR,CESM2,historical,r1i1p1f1,day,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
4,CMIP6,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...


## 8. Directly access the results
Once the index is queried the results are stored in a Pandas `DataFrame`. This `DataFrame` can be accessed by the `df` property. The `DataFrame` can be further refined and passed back to the catalog, updating the contents. 

In [9]:
result.df[result.df.table_id=="day"].groupby(["institution_id", "table_id", "member_id"]).nunique()

mip_era  activity_drs  source_id  \
institution_id table_id member_id                                     
MIROC          day      r10i1p1f1        1             1          1   
                        r11i1p1f1        1             1          1   
                        r12i1p1f1        1             1          1   
                        r13i1p1f1        1             1          1   
                        r14i1p1f1        1             1          1   
...                                    ...           ...        ...   
NCAR           day      r5i1p1f1         1             1          1   
                        r6i1p1f1         1             1          1   
                        r7i1p1f1         1             1          1   
                        r8i1p1f1         1             1          1   
                        r9i1p1f1         1             1          1   

                                   experiment_id  variable_id  grid_label  \
institution_id table_id member_id                                           
MIROC          day      r10i1p1f1              1            1           1   
                        r11i1p1f1              1            1           1   
                        r12i1p1f1              1            1           1   
                        r13i1p1f1              1            1           1   
                        r14i1p1f1              1            1           1   
...                                          ...          ...         ...   
NCAR           day      r5i1p1f1               1            1           1   
                        r6i1p1f1               1            1           1   
                        r7i1p1f1               1            1           1   
                        r8i1p1f1               1            1           1   
                        r9i1p1f1               1            1           1   

                                   version  path  
institution_id table_id member_id                 
MIROC          day      r10i1p1f1        1     1  
                        r11i1p1f1        1     1  
                        r12i1p1f1        1     1  
                        r13i1p1f1        1     1  
                        r14i1p1f1        1     1  
...                                    ...   ...  
NCAR           day      r5i1p1f1         1     1  
                        r6i1p1f1         1     1  
                        r7i1p1f1         1     1  
                        r8i1p1f1         1     1  
                        r9i1p1f1         1     1  

[61 rows x 8 columns]

### Filter by daily data and find common member_id's

In [10]:
source_members = [x for idx, x in result.df[result.df.table_id=="day"].groupby(["source_id"])]
common_members = [set(x["member_id"]) for x in source_members]
common_members = common_members[0].intersection(*common_members[1:])
common_members

{'r10i1p1f1',
 'r11i1p1f1',
 'r1i1p1f1',
 'r2i1p1f1',
 'r3i1p1f1',
 'r4i1p1f1',
 'r5i1p1f1',
 'r6i1p1f1',
 'r7i1p1f1',
 'r8i1p1f1',
 'r9i1p1f1'}

### Check for multiple versions

In [11]:
common_day = result.df[(result.df.table_id=='day') & result.df.member_id.isin(common_members)]
common_day.groupby(["institution_id", "source_id", "member_id"]).nunique()

mip_era  activity_drs  experiment_id  \
institution_id source_id member_id                                         
MIROC          MIROC6    r10i1p1f1        1             1              1   
                         r11i1p1f1        1             1              1   
                         r1i1p1f1         1             1              1   
                         r2i1p1f1         1             1              1   
                         r3i1p1f1         1             1              1   
                         r4i1p1f1         1             1              1   
                         r5i1p1f1         1             1              1   
                         r6i1p1f1         1             1              1   
                         r7i1p1f1         1             1              1   
                         r8i1p1f1         1             1              1   
                         r9i1p1f1         1             1              1   
NCAR           CESM2     r10i1p1f1        1             1              1   
                         r11i1p1f1        1             1              1   
                         r1i1p1f1         1             1              1   
                         r2i1p1f1         1             1              1   
                         r3i1p1f1         1             1              1   
                         r4i1p1f1         1             1              1   
                         r5i1p1f1         1             1              1   
                         r6i1p1f1         1             1              1   
                         r7i1p1f1         1             1              1   
                         r8i1p1f1         1             1              1   
                         r9i1p1f1         1             1              1   

                                    table_id  variable_id  grid_label  \
institution_id source_id member_id                                      
MIROC          MIROC6    r10i1p1f1         1            1           1   
                         r11i1p1f1         1            1           1   
                         r1i1p1f1          1            1           1   
                         r2i1p1f1          1            1           1   
                         r3i1p1f1          1            1           1   
                         r4i1p1f1          1            1           1   
                         r5i1p1f1          1            1           1   
                         r6i1p1f1          1            1           1   
                         r7i1p1f1          1            1           1   
                         r8i1p1f1          1            1           1   
                         r9i1p1f1          1            1           1   
NCAR           CESM2     r10i1p1f1         1            1           1   
                         r11i1p1f1         1            1           1   
                         r1i1p1f1          1            1           1   
                         r2i1p1f1          1            1           1   
                         r3i1p1f1          1            1           1   
                         r4i1p1f1          1            1           1   
                         r5i1p1f1          1            1           1   
                         r6i1p1f1          1            1           1   
                         r7i1p1f1          1            1           1   
                         r8i1p1f1          1            1           1   
                         r9i1p1f1          1            1           1   

                                    version  path  
institution_id source_id member_id                 
MIROC          MIROC6    r10i1p1f1        1     1  
                         r11i1p1f1        1     1  
                         r1i1p1f1         1     1  
                         r2i1p1f1         1     1  
                         r3i1p1f1         1     1  
                         r4i1p1f1         1     1  
                         r5i1p1f1         

### Update the catalog with the new filtered results

In [12]:
result.df = common_day.reset_index(drop=True)
result

,mip_era,activity_drs,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,path
0,CMIP6,CMIP,MIROC,MIROC6,historical,r2i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
1,CMIP6,CMIP,MIROC,MIROC6,historical,r5i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
2,CMIP6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
3,CMIP6,CMIP,MIROC,MIROC6,historical,r4i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
4,CMIP6,CMIP,MIROC,MIROC6,historical,r3i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
5,CMIP6,CMIP,MIROC,MIROC6,historical,r9i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
6,CMIP6,CMIP,MIROC,MIROC6,historical,r8i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
7,CMIP6,CMIP,MIROC,MIROC6,historical,r10i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
8,CMIP6,CMIP,MIROC,MIROC6,historical,r7i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
9,CMIP6,CMIP,MIROC,MIROC6,historical,r11i1p1f1,day,tas,gn,v20200519,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...


### List the unique keys
These keys can be used as index values for the catalog e.g. `result['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.day.tas.gn.v20191016']`

In [13]:
list(result)

['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r5i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r1i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r4i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r3i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r9i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r8i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r10i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r7i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r11i1p1f1.day.tas.gn.v20200519',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r6i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.NCAR.CESM2.historical.r8i1p1f1.day.tas.gn.v20190311',
 'CMIP6.CMIP.NCAR.CESM2.historical.r10i1p1f1.day.tas.gn.v20190313',
 'CMIP6.CMIP.NCAR.CESM2.historical.r7i1p1f1.day.tas.gn.v20190311',
 'CMIP6.CMIP.NCAR.CESM2.historical.r1

## 9. Change how the datasets are opened
By default the datasets are opened using `open_mfdataset` from either `xarray` or `xcdat`.

The default arguments to open the datsets are `parallel=True` and `chunks='auto'`.

These settings can be changed by recreating the catalog.

In [14]:
mutated_result = result(xarray_kwargs={"chunks": {"time": 100}})
mutated_result

,mip_era,activity_drs,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,path
0,CMIP6,CMIP,MIROC,MIROC6,historical,r2i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
1,CMIP6,CMIP,MIROC,MIROC6,historical,r5i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
2,CMIP6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
3,CMIP6,CMIP,MIROC,MIROC6,historical,r4i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
4,CMIP6,CMIP,MIROC,MIROC6,historical,r3i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
5,CMIP6,CMIP,MIROC,MIROC6,historical,r9i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
6,CMIP6,CMIP,MIROC,MIROC6,historical,r8i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
7,CMIP6,CMIP,MIROC,MIROC6,historical,r10i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
8,CMIP6,CMIP,MIROC,MIROC6,historical,r7i1p1f1,day,tas,gn,v20191016,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
9,CMIP6,CMIP,MIROC,MIROC6,historical,r11i1p1f1,day,tas,gn,v20200519,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...


## 10. Opening the data
For single entry in the catalog a dataset can be opened with `to_xarray` or `to_xcdat`

In [15]:
ds = mutated_result['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.day.tas.gn.v20191016'].to_xarray()
ds

<xarray.Dataset>
Dimensions:    (time: 60265, bnds: 2, lat: 128, lon: 256)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(100, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(3652, 128, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(3652, 256, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(100, 128, 256), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  10958.0
    creation_date:          2019-08-30T10:41:53Z
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            tas
    variant_label:          r2i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/ae3271df-3875-4909-a42f-878606c0200b

In [16]:
ds = mutated_result['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.day.tas.gn.v20191016'].to_xcdat()
ds

<xarray.Dataset>
Dimensions:    (time: 60265, bnds: 2, lat: 128, lon: 256)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(100, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(3652, 128, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(3652, 256, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(100, 128, 256), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  10958.0
    creation_date:          2019-08-30T10:41:53Z
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            tas
    variant_label:          r2i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/ae3271df-3875-4909-a42f-878606c0200b

### Open all the datasets
All the datasets can be opened using `to_dataset_dict` or `to_datatree`

In [17]:
datasets = mutated_result.to_dataset_dict()
list(datasets)

['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r5i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r1i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r4i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r3i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r9i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r8i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r10i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r7i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r11i1p1f1.day.tas.gn.v20200519',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r6i1p1f1.day.tas.gn.v20191016',
 'CMIP6.CMIP.NCAR.CESM2.historical.r8i1p1f1.day.tas.gn.v20190311',
 'CMIP6.CMIP.NCAR.CESM2.historical.r10i1p1f1.day.tas.gn.v20190313',
 'CMIP6.CMIP.NCAR.CESM2.historical.r7i1p1f1.day.tas.gn.v20190311',
 'CMIP6.CMIP.NCAR.CESM2.historical.r1

In [18]:
dt = mutated_result.to_datatree()
dt

DataTree('None', parent=None)
└── DataTree('CMIP6')
    └── DataTree('CMIP')
        ├── DataTree('MIROC')
        │   └── DataTree('MIROC6')
        │       └── DataTree('historical')
        │           ├── DataTree('r2i1p1f1')
        │           │   └── DataTree('day')
        │           │       └── DataTree('tas')
        │           │           └── DataTree('gn')
        │           │               └── DataTree('v20191016')
        │           │                       Dimensions:    (time: 60265, bnds: 2, lat: 128, lon: 256)
        │           │                       Coordinates:
        │           │                         * time       (time) object 1850-01-01 12:00:00 ... 2014-12-31 12:00:00
        │           │                         * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
        │           │                         * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
        │           │                           height     float64 2.0
        │           │                       Dimensions without coordinates: bnds
        │           │                       Data variables:
        │           │                           time_bnds  (time, bnds) object dask.array<chunksize=(100, 2), meta=np.ndarray>
        │           │                           lat_bnds   (lat, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
        │           │                           lon_bnds   (lon, bnds) float64 dask.array<chunksize=(256, 2), meta=np.ndarray>
        │           │                           tas        (time, lat, lon) float32 dask.array<chunksize=(100, 128, 256), meta=np.ndarray>
        │           │                       Attributes: (12/44)
        │           │                           Conventions:            CF-1.7 CMIP-6.2
        │           │                           activity_id:            CMIP
        │           │                           branch_method:          standard
        │           │                           branch_time_in_child:   0.0
        │           │                           branch_time_in_parent:  10958.0
        │           │                           creation_date:          2019-08-30T10:41:53Z
        │           │                           ...                     ...
        │           │                           title:                  MIROC6 output prepared for CMIP6
        │           │                           variable_id:            tas
        │           │                           variant_label:          r2i1p1f1
        │           │                           license:                CMIP6 model data produced by MIROC is licensed un...
        │           │                           cmor_version:           3.4.0
        │           │                           tracking_id:            hdl:21.14100/ae3271df-3875-4909-a42f-878606c0200b
        │           ├── DataTree('r5i1p1f1')
        │           │   └── DataTree('day')
        │           │       └── DataTree('tas')
        │           │           └── DataTree('gn')
        │           │               └── DataTree('v20191016')
        │           │                       Dimensions:    (time: 60265, bnds: 2, lat: 128, lon: 256)
        │           │                       Coordinates:
        │           │                         * time       (time) object 1850-01-01 12:00:00 ... 2014-12-31 12:00:00
        │           │                         * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
        │           │                         * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
        │           │                           height     float64 2.0
        │           │                       Dimensions without coordinates: bnds
        │           │                       Data variables:
        │           │                           time_bnds  (time, bnds) object dask.array<chunksize=(100,